<a href="https://colab.research.google.com/github/szyxxx/MachineLearning-TelkomUniversity/blob/main/Modern-CNN/DenseNet/AxelDavid_1103210017_TK4504_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Axel David<br>
NIM: 1103210017<br>
Judul Tugas: Pembuatan Model CNN Modern - DenseNet<br>
Lecture: 12<br>
Model: DenseNet<br>
Dataset: CIFAR-10

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import tensorflow as tf
from PIL import Image

In [ ]:
# Mengunduh dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalisasi dataset
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Konversi label ke tensor
train_labels = torch.tensor(train_labels, dtype=torch.long).squeeze()
test_labels = torch.tensor(test_labels, dtype=torch.long).squeeze()

# Membuat transformasi
transform = transforms.Compose([
    transforms.ToPILImage(),  # Mengonversi gambar menjadi PIL Image
    transforms.Resize(224),  # Mengubah ukuran gambar menjadi 224x224
    transforms.ToTensor(),  # Mengonversi gambar menjadi tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalisasi gambar
])

# Membuat dataset CIFAR-10
class CIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

trainset = CIFAR10Dataset(train_images, train_labels, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = CIFAR10Dataset(test_images, test_labels, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

In [ ]:
# Mendefinisikan blok Dense
class Bottleneck(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(self.bn1(x))
        out = self.conv2(self.bn2(out))
        out = torch.cat([out, x], 1)
        return out

# Mendefinisikan blok Transition
class Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2)

    def forward(self, x):
        out = self.conv(self.bn(x))
        out = self.pool(out)
        return out

# Mendefinisikan arsitektur DenseNet
class DenseNet(nn.Module):
    def __init__(self, num_blocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        num_planes = 2 * growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(Bottleneck, num_planes, num_blocks[0], growth_rate)
        num_planes += num_blocks[0] * growth_rate
        out_planes = int(math.floor(num_planes * reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(Bottleneck, num_planes, num_blocks[1], growth_rate)
        num_planes += num_blocks[1] * growth_rate
        out_planes = int(math.floor(num_planes * reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(Bottleneck, num_planes, num_blocks[2], growth_rate)
        num_planes += num_blocks[2] * growth_rate
        out_planes = int(math.floor(num_planes * reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(Bottleneck, num_planes, num_blocks[3], growth_rate)
        num_planes += num_blocks[3] * growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_channels, nblock, growth_rate):
        layers = []
        for i in range(nblock):
            layers.append(block(in_channels, growth_rate))
            in_channels += growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = torch.squeeze(out)
        out = self.bn(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

def densenet121():
    return DenseNet(num_blocks=[6, 12, 24, 16], growth_rate=32)

net_densenet = densenet121()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net_densenet.parameters(), lr=0.01, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_densenet.to(device)

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net_densenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Cetak setiap 10 batch
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0

print("Finished Training")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net_densenet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')